In [33]:
import torch
from torch import nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


import numpy as np
from tqdm import tqdm

from model import create_transformer_model


In [5]:
en_raw = "../data/english.txt"
fr_raw = "../data/french.txt"

In [ ]:
START_TOKEN = '<SOS>'
PADDING_TOKEN = '<PAD>' # Used for padding and ignored in loss
END_TOKEN = '<EOS>'
UNKNOWN_TOKEN = '<UNK>' # For handling unknown chars during evaluation


In [7]:
base_french_chars = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                     ':', '<', '=', '>', '?', '@', 
                     'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                     'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                     'Y', 'Z',
                     '[', '\\', ']', '^', '_', '`', 
                     'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                     'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                     'y', 'z', 
                     '{', '|', '}', '~',
                     'à', 'â', 'ä', 'æ', 'ç', 'é', 'è', 'ê', 'ë', 'î', 'ï', 'ô', 'œ', 'ù', 'û', 'ü', 'ÿ',
                     'À', 'Â', 'Ä', 'Æ', 'Ç', 'É', 'È', 'Ê', 'Ë', 'Î', 'Ï', 'Ô', 'Œ', 'Ù', 'Û', 'Ü', 'Ÿ',
                     ]

In [8]:
base_english_chars = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@', 
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                        'Y', 'Z',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~']

In [9]:
# PADDING_TOKEN usually gets index 0
french_vocabulary = [PADDING_TOKEN, START_TOKEN, END_TOKEN, UNKNOWN_TOKEN] + base_french_chars
english_vocabulary = [PADDING_TOKEN, START_TOKEN, END_TOKEN, UNKNOWN_TOKEN] + base_english_chars


In [10]:
with open(en_raw, 'r') as file:
    en_sentences = file.readlines()
with open(fr_raw, 'r') as file:
    fr_sentences = file.readlines()

In [11]:
en_sentences = [sentence.rstrip('\n') for sentence in en_sentences]
fr_sentences = [sentence.rstrip('\n') for sentence in fr_sentences]

# Removes the narrow non-breaking space (\u202f) and replace it with a normal space
fr_sentences = [sentence.replace('\u202f', ' ') for sentence in fr_sentences]


In [12]:
print(f"English: {en_sentences[:10]} \nFrench: {fr_sentences[:10]}\n")

English: ['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.'] 
French: ['Va !', 'Salut !', 'Salut.', 'Cours !', 'Courez !', 'Qui ?', 'Ça alors !', 'Au feu !', "À l'aide !", 'Saute.']



In [13]:
index_to_fr = {k:v for k,v in enumerate(french_vocabulary)}
fr_to_index = {v:k for k,v in enumerate(french_vocabulary)}
index_to_en = {k:v for k,v in enumerate(english_vocabulary)}
en_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [16]:
# --- Define Padding Index Constant ---
# IMPORTANT: This index must match the PADDING_TOKEN position in vocab
IGNORE_IDX = fr_to_index[PADDING_TOKEN] # Used for loss function
EN_PADDING_IDX = en_to_index[PADDING_TOKEN] # Used for padding source batches
FR_PADDING_IDX = fr_to_index[PADDING_TOKEN] # Used for padding target batches
# Verify these are likely 0 if PADDING_TOKEN is first in vocab list
print(f"Padding Index (Ignore Loss / FR Pad): {IGNORE_IDX}")
print(f"EN Padding Index: {EN_PADDING_IDX}")

Padding Index (Ignore Loss / FR Pad): 0
EN Padding Index: 0


In [17]:
#TOTAL_SENTENCES = 400000
en = en_sentences[:]
fr = fr_sentences[:]

In [18]:
print(f"{len(en)} sentences")
print(en[:10])
print(fr[:10])

170651 sentences
['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.']
['Va !', 'Salut !', 'Salut.', 'Cours !', 'Courez !', 'Qui ?', 'Ça alors !', 'Au feu !', "À l'aide !", 'Saute.']


In [19]:
max(len(x) for x in fr), max(len(x) for x in en),

(349, 286)

In [20]:
PERCENTILE = 99
print( f"{PERCENTILE}th percentile length french: {np.percentile([len(x) for x in fr], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in en], PERCENTILE)}" )


99th percentile length french: 85.0
99th percentile length English: 70.0


In [21]:
max_sequence_length = 100

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(fr)):
    french_sentence, english_sentence = fr[index], en[index]
    if is_valid_length(french_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(french_sentence, french_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(fr)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 170651
Number of valid sentences: 164317


In [22]:
french = [fr[i] for i in valid_sentence_indicies]
english = [en[i] for i in valid_sentence_indicies]

In [23]:
len(english)
#164317

164317

In [24]:
french[::50000]

['Va !',
 "Quiconque l'a-t-il remarqué ?",
 'Nous parlâmes de divers sujets.',
 'Le match fut annulé à cause de la pluie.']

In [25]:
english[::50000]

['Go.',
 'Did anybody notice this?',
 'We talked about various topics.',
 'The game was called off on account of the rain.']

In [ ]:
transformer = create_transformer_model(
    src_vocab_size=len(english_vocabulary),
    tgt_vocab_size=len(french_vocabulary),
    d_model=512,
    num_heads=8,
    num_encoder_layers=3,
    num_decoder_layers=3,
    d_ff=2048,
    max_seq_len= 100,
    dropout=0.1
)

In [27]:
transformer

Transformer(
  (src_embedding): Embedding(98, 512)
  (tgt_embedding): Embedding(132, 512)
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-2): 3 x EncoderLayer(
      (self_attention): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): 

In [60]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Trainable parameters: {count_parameters(transformer):,}")


Trainable parameters: 22,254,724


In [28]:
from sklearn.model_selection import train_test_split

filtered_english = [en[i] for i in valid_sentence_indicies]
filtered_french = [fr[i] for i in valid_sentence_indicies]

# 90% train, 10% validation
train_english, val_english, train_french, val_french = train_test_split(
    filtered_english, filtered_french, test_size=0.1, random_state=42 
)
print(f"Training samples: {len(train_english)}, Validation samples: {len(val_english)}")


Training samples: 147885, Validation samples: 16432


In [34]:
def tokenize(sentence, vocab):
    return [char for char in sentence if char in vocab]

class TextDataset(Dataset): 
    def __init__(self, english, french, en_to_index, fr_to_index):
        self.english = english
        self.french = french
        self.en_to_index = en_to_index
        self.fr_to_index = fr_to_index

    def __len__(self):
        return len(self.english)

    def __getitem__(self, idx):
        eng_sentence, french_sentence = self.english[idx], self.french[idx]

        eng_tokenized = [START_TOKEN] + tokenize(eng_sentence, english_vocabulary) + [END_TOKEN]
        fr_tokenized = [START_TOKEN] + tokenize(french_sentence, french_vocabulary) + [END_TOKEN]

        eng_numericalized = [self.en_to_index[token] for token in eng_tokenized]
        fr_numericalized = [self.fr_to_index[token] for token in fr_tokenized]
        return torch.tensor(eng_numericalized), torch.tensor(fr_numericalized)

In [35]:
train_dataset = TextDataset(train_english, train_french, en_to_index, fr_to_index)
val_dataset = TextDataset(val_english, val_french, en_to_index, fr_to_index)


In [36]:
def padding_batch(batch, en_padding_idx, fr_padding_idx): # Accept indices
    src_list, tgt_list = zip(*batch)
    # Check if lists are empty before calculating max
    if not src_list or not tgt_list:
         return torch.empty(0), torch.empty(0) # Or handle as appropriate

    src_max_len = max(src.size(0) for src in src_list) if src_list else 0
    tgt_max_len = max(tgt.size(0) for tgt in tgt_list) if tgt_list else 0

    padded_src_list = []
    padded_tgt_list = []
    for src, tgt in zip(src_list, tgt_list):
        pad_len_src = src_max_len - src.size(0)
        pad_len_tgt = tgt_max_len - tgt.size(0)
        # Use the passed indices directly
        padded_src = F.pad(src, (0, pad_len_src), value=en_padding_idx)
        padded_tgt = F.pad(tgt, (0, pad_len_tgt), value=fr_padding_idx)
        padded_src_list.append(padded_src)
        padded_tgt_list.append(padded_tgt)

    # Only stack if lists are not empty
    final_src = torch.stack(padded_src_list) if padded_src_list else torch.empty(0)
    final_tgt = torch.stack(padded_tgt_list) if padded_tgt_list else torch.empty(0)
    return final_src, final_tgt


In [37]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=lambda batch: padding_batch(batch, EN_PADDING_IDX, FR_PADDING_IDX),
                          drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=lambda batch: padding_batch(batch, EN_PADDING_IDX, FR_PADDING_IDX),
                        drop_last=True)

In [38]:
criterion = nn.CrossEntropyLoss(ignore_index=IGNORE_IDX) # Use the defined constant
optim = torch.optim.AdamW(transformer.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
clip_grad_norm = 1.0


In [ ]:
def loss_fn(output, target, criterion):

    # Output: [batch_size * seq_len, vocab_size]
    # Target: [batch_size * seq_len]
    output_flat = output.contiguous().view(-1, output.shape[-1])
    target_flat = target.contiguous().view(-1)

    # Criterion handles ignoring padding index and reduction (usually mean)
    loss = criterion(output_flat, target_flat)
    return loss



In [ ]:
def train_loop():
    transformer.train()
    transformer.to(device)
    #total_loss = 0
    num_epochs = 2 


    print(f"Training on device: {device}") 

    for epoch in range(num_epochs):
        print(f"--- Epoch {epoch+1}/{num_epochs} ---")
        transformer.train() 
        total_epoch_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1} Training')

        for batch_num, batch in enumerate(progress_bar):
            english_batch, french_batch = batch
            english_batch = english_batch.to(device) 
            french_batch = french_batch.to(device) 

            optim.zero_grad()

            # 2. Forward Pass 
            output = transformer(english_batch, french_batch)
            # output shape: [batch_size, tgt_len-1, tgt_vocab_size]

            loss = loss_fn(output, french_batch[:, 1:], criterion)

            # 4. Backward Pass
            loss.backward()

            # 5. Gradient Clipping
            torch.nn.utils.clip_grad_norm_(transformer.parameters(), clip_grad_norm)

            # 6. Optimizer Step
            optim.step()

            total_epoch_loss += loss.item()
            progress_bar.set_postfix({'batch_loss': f'{loss.item():.4f}'})

            if batch_num > 0 and batch_num % 200 == 0: 
                print(f"\n  Batch {batch_num} Loss: {loss.item():.4f}")
                transformer.eval()
                with torch.no_grad():
  
                    input_sentence = "should we go to the mall?" # a fixed sentence for eval
                    input_tokenized = [START_TOKEN] + tokenize(input_sentence, english_vocabulary) + [END_TOKEN]
                    input_numericalized = [en_to_index.get(token, en_to_index.get(UNKNOWN_TOKEN, 0)) for token in input_tokenized]
                    input_tensor = torch.tensor(input_numericalized).unsqueeze(0).to(device)

                    start_symbol_idx = fr_to_index.get(START_TOKEN)
                    if start_symbol_idx is None:
                       raise ValueError("START_TOKEN not found in fr_to_index mapping")

                    output_indices = transformer.greedy_decode(input_tensor, max_len=max_sequence_length, start_symbol=start_symbol_idx)[0].cpu().numpy()

                  
                    pad_idx = fr_to_index.get(PADDING_TOKEN)
                    end_idx = fr_to_index.get(END_TOKEN)

                    predicted_sentence_chars = [
                        index_to_fr[idx] for idx in output_indices
                        if idx != pad_idx and idx != start_symbol_idx and idx != end_idx and idx in index_to_fr
                    ]
                    predicted_sentence = "".join(predicted_sentence_chars)
                    print(f"  Eval (should we go...): {predicted_sentence}")
                    print("  -------------------------------------------")
                transformer.train() # Set back to train mode

        avg_train_loss = total_epoch_loss / len(train_loader)
        avg_val_loss = val_loop() 
        print(f"Epoch {epoch+1} Summary: Avg Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")



def val_loop(): # Validation loop
    transformer.eval()
    total_val_loss = 0
    with torch.no_grad():
        progress_bar_val = tqdm(val_loader, desc=f'Validation')
        for batch_idx, batch in enumerate(progress_bar_val):
            english_batch, french_batch = batch
            english_batch = english_batch.to(device)
            french_batch = french_batch.to(device)

            output = transformer(english_batch, french_batch)
            val_loss = loss_fn(output, french_batch[:, 1:], criterion) 

            total_val_loss += val_loss.item()
            progress_bar_val.set_postfix({'val_loss': val_loss.item()})

    avg_val_loss = total_val_loss / len(val_loader)
    return avg_val_loss


if __name__ == "__main__":
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu') 
    train_loop() 

Training on device: mps
--- Epoch 1/2 ---


Epoch 1 Training:   4%|▍         | 200/4621 [03:54<38:32,  1.91it/s, batch_loss=2.1278]  


  Batch 200 Loss: 2.1278


Epoch 1 Training:   4%|▍         | 201/4621 [04:17<9:07:20,  7.43s/it, batch_loss=2.1278]

  Eval (should we go...): Quellle de de de de de de ?re ?re ?le ?le ?
  -------------------------------------------


...


  Batch 4600 Loss: 0.7967


Epoch 2 Training: 100%|█████████▉| 4601/4621 [42:55<00:39,  1.98s/it, batch_loss=0.7967]

  Eval (should we go...): Devrions-nous aller le mal ? ? ? ? ? ?
  -------------------------------------------


Validation: 100%|██████████| 513/513 [01:28<00:00,  5.78it/s, val_loss=0.646]

Epoch 2 Summary: Avg Train Loss: 0.8604, Avg Val Loss: 0.6674


In [64]:
transformer.eval()
def translate(eng_sentence_text): 
    """Translates an English sentence to French using greedy decoding."""
    with torch.no_grad(): # Disable gradients during inference
      
        input_tokenized = [START_TOKEN] + tokenize(eng_sentence_text, english_vocabulary) + [END_TOKEN] # Tokenize input sentence
        input_numericalized = [en_to_index[token] for token in input_tokenized] # Numericalize input tokens
        input_tensor = torch.tensor(input_numericalized).unsqueeze(0).to(device) # Create tensor, add batch dimension, move to device

        # Greedy Decode 
        output_indices = transformer.greedy_decode(
            input_tensor, max_len=max_sequence_length, start_symbol=fr_to_index[START_TOKEN]
        )[0].cpu().numpy() 

        # 3. Convert Output Indices to French Sentence
        predicted_sentence_chars = []
        for idx in output_indices:
            if idx == fr_to_index[END_TOKEN]: 
                break
            predicted_sentence_chars.append(index_to_fr[idx]) 

        predicted_sentence = "".join(predicted_sentence_chars) 
        return predicted_sentence



In [65]:
english_input = "what should we do when the day starts?"
french_translation = translate(english_input)
print(f"English Input: {english_input}")
print(f"French Translation: {french_translation}")

English Input: what should we do when the day starts?
French Translation: <SOS>Ce que devrais-nous faire le dire ?


In [66]:
english_input = "what do you think about the book?"
french_translation = translate(english_input)
print(f"English Input: {english_input}")
print(f"French Translation: {french_translation}")

English Input: what do you think about the book?
French Translation: <SOS>Ce que vous pensez-vous ?


In [67]:
english_input = "where are you going?"
french_translation = translate(english_input)
print(f"English Input: {english_input}")
print(f"French Translation: {french_translation}")

English Input: where are you going?
French Translation: <SOS>Où vous avez en train de matin ?


In [68]:
english_input = "I want a new book."
french_translation = translate(english_input)
print(f"English Input: {english_input}")
print(f"French Translation: {french_translation}")

English Input: I want a new book.
French Translation: <SOS>Je veux un livre un livre.
